# IBM Capstone Course
### Week 3 Peer-graded Assignment

In this assignment, we will explore the neighborhoods in the city of Toronto.

### Importing Libraries

In [37]:
from bs4 import BeautifulSoup as bsoup
from urllib.request import urlopen as uReq
import requests
import lxml
import pandas as pd
from pandas import DataFrame
import numpy as np

### Scaping and Parsing Data

In [122]:
#url of the page to be scraped
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

req=requests.get(url)

#parse the table with soup using html parser
page=bsoup(req.text,"html.parser")

#extracting the table from the data
t=page.table
results=t.find_all('tr')

In [123]:
table = []
for row in results:
    info = row.text.split('\n')[1:-1] # remove empty str (the first and last items)
    table.append(info)

### Converting Table into a Pandas Dataframe

In [257]:
df = pd.DataFrame(table[1:], columns=table[0])
df.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Data Cleaning and Sorting

#### Dropping Cells with "Unassigned" Boroughs

In [258]:
#dropping rows where values in the Borough column are "Not Assigned"
df = df[df.Borough != 'Not assigned']

#### Correcting the Index of the Data Frame

In [259]:
#Correcting the index of the data frame
df.reset_index(drop=True, inplace=True)

#### Correcting Column Names

In [260]:
#Correcting column names
df.rename(columns={'Postcode':'Postal Code'}, inplace=True)
df.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
df.head(5)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### If a row has a Borough but an unassigned Neighbourhood then the Neighbourhood will be the same as the Borough

In [261]:
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']
df.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### For multiple neighbourhoods per Postal Code, the neighbourhood will be combined in one row separated by a comma

In [267]:
df = df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head(5)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Shape of the Data Frame

In [263]:
df.shape

(103, 3)

# Part: 2 - Attach Latitude and Longitude to the Dataframe

In [264]:
geo_data = pd.read_csv("Geospatial_Coordinates.csv")
geo_data.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merging

In [269]:
new_df = df.merge(geo_data, on = "Postal Code")
new_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [229]:
#new_df.to_csv("toronto_base.csv")

# Explore and Cluster the Neighbourhoods of Toronto
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

    to add enough Markdown cells to explain what you decided to do and to report any observations you make.
    to generate maps to visualize your neighborhoods and how they cluster together. 

In [143]:
#Installing and importing libraries
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from sklearn.cluster import KMeans
import geocoder
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim  #pip install geopy

In [270]:
new_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Let's Create Toronto's Map using Folium

In [271]:
city = 'Toronto, ON'

geo_locator = Nominatim(user_agent="Toronto")
loc = geo_locator.geocode(city)
latitude = loc.latitude
longitude = loc.longitude
print('Latitude: \t', latitude)
print('Longitude: \t', longitude)

Latitude: 	 43.653963
Longitude: 	 -79.387207


In [272]:
toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, Neighbourhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['Borough'], new_df['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7
        ).add_to(toronto)  
    
toronto

### Number of Neighbourhoods for each Borough

In [273]:
new_df['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

North York has the most Neighborhoods followed by Downtown Toronto and Scarborough.

### Let's Explore Neighborhoods in North York

In [274]:
northyork_data = new_df[new_df['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493
5,M2N,North York,Willowdale South,43.770120,-79.408493
6,M2P,North York,York Mills West,43.752758,-79.400049
7,M2R,North York,Willowdale West,43.782736,-79.442259
8,M3A,North York,Parkwoods,43.753259,-79.329656
9,M3B,North York,Don Mills North,43.745906,-79.352188


In [354]:
#Latitude and Longitide for North York
borough_ny = 'North York,Toronto'
latitude_ny = 43.773077
longitude_ny = -79.257774
print('Latitude of North York,Toronto: \t', latitude_ny)
print('Longitude of North York,Toronto: \t', longitude_ny)

Latitude of North York,Toronto: 	 43.773077
Longitude of North York,Toronto: 	 -79.257774


In [245]:
#Map of North York and it's Neighborhoods
ny_map = folium.Map(location=[latitude_ny, longitude_ny], zoom_start=12)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(ny_map)  
    
ny_map

### Let's look at Top 200 Venues in North York using Foursqaure API

In [364]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [356]:
radius = 500
limit = 200
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d1ef7049df1dc7a948200f5'},
 'response': {'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.775537,
          'lng': -79.256833}],
        'distance

In [357]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [280]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
3,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
4,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
5,DAVIDsTEA,Tea Room,43.776613,-79.258516
6,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
7,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
8,Hot Topic,Clothing Store,43.775450,-79.257929
9,Tim Hortons,Coffee Shop,43.769936,-79.260368


In [358]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Venues for each Neighborhood in North York
The cell below tells us about the neighbourhoods in North York

In [284]:
northyork_venues = getNearbyVenues(names=northyork_data['Neighborhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Downsview, North Park, Upwood Park
Humber Summit
Emery, Humberlea


### Number of Venues in each Neighborhood in North York
This gives us an idea regarding the relative popularity, commercialization, and tourism pull etc foe each neighborhood

In [332]:
northyork_venues.Neighborhood.value_counts()

Willowdale South                                   30
Fairview, Henry Farm, Oriole                       30
Bedford Park, Lawrence Manor East                  25
Flemingdon Park, Don Mills South                   23
Bathurst Manor, Downsview North, Wilson Heights    18
Lawrence Heights, Lawrence Manor                   16
Northwood Park, York University                     7
York Mills West                                     5
Don Mills North                                     5
Downsview, North Park, Upwood Park                  5
Willowdale West                                     5
Downsview Northwest                                 5
Bayview Village                                     4
Glencairn                                           4
Victoria Village                                    4
Hillcrest Village                                   4
Downsview Central                                   4
CFB Toronto, Downsview East                         3
Humber Summit               

In [359]:
group = northyork_venues[['Neighborhood','Venue']]
group.set_index(['Neighborhood'])
group.set_index(['Neighborhood'])

,Venue
Neighborhood,
Hillcrest Village,Eagle's Nest Golf Club
Hillcrest Village,AY Jackson Pool
Hillcrest Village,Villa Madina
Hillcrest Village,Duncan Creek Park
"Fairview, Henry Farm, Oriole",The LEGO Store
"Fairview, Henry Farm, Oriole",SilverCity Fairview Mall Cinemas
"Fairview, Henry Farm, Oriole",CF Fairview Mall
"Fairview, Henry Farm, Oriole",Michel's Baguette
"Fairview, Henry Farm, Oriole",Hero Certified Burgers


### One-hot Encoding
Since the data is categorical data, we need to use one-hot encoding to apply any clustering algorithm on it. Clustering algorithm do not work on categorical data

In [360]:
# one hot encoding
ny_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Line,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fraternity House,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Other Repair Shop,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoke Shop,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [333]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped.head(7)

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Line,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fraternity House,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Other Repair Shop,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoke Shop,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.055556,0.0,0.00,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.111111,0.00,0.0,0.0,0.0,0.00,0.055556,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.00,0.0,0.055556,0.055556,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.000000,0.000000,0.0,0.055556,0.055556,0.0,0.0,0.0,0.00,0.0,0.055556,0.0,0.055556,0.055556,0.0,0.0,0.0,0.0,0.055556,0.055556,0.0,0.00,0.0,0.0,0.055556,0.0,0.0
1,Bayview Village,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.250000,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.25,0.0,0.0,0.25,0.0,0.000000,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.25,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.04,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.04,0.0,0.04,0.0,0.0,0.00,0.0,0.080000,0.04,0.0,0.0,0.0,0.04,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.080000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.04,0.04,0.0,0.0,0.0,0.00,0.04,0.04,0.0,0.0,0.08,0.04,0.04,0.00,0.04,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.040000,0.040000,0.0,0.0,0.0,0.04,0.0,0.040000,0.0,0.040000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.040000,0.0,0.04,0.0,0.0,0.000000,0.0,0.0
3,"CFB Toronto, Downsview East",0.0,0.333333,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.333333,0.333333,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0
4,Don Mills North,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.20,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.20,0.0,0.2,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.2,0.0,0.00,0.00,0.00,0.0

### K-Means Clustering

In [361]:
ny_data = northyork_data.drop(16)
# set number of clusters
kclusters = 5

ny_clustering = ny_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 0, 3, 3, 3, 3, 3, 4])

In [362]:
ny_merged = ny_data

# add clustering labels
ny_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(group.set_index('Neighborhood'), on='Neighborhood')

ny_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,3,Eagle's Nest Golf Club
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,3,AY Jackson Pool
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,3,Villa Madina
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,3,Duncan Creek Park
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,3,The LEGO Store
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,3,SilverCity Fairview Mall Cinemas
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,3,CF Fairview Mall
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,3,Michel's Baguette
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,3,Hero Certified Burgers
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,3,Shoppers Drug Mart


### Plotting of Clusters 

In [363]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_ny, longitude_ny], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Explore each Cluster

In [353]:
new = ny_merged.set_index(['Cluster Labels'])
new.sort_index()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Venue
Cluster Labels,,,,,,
0,M9M,North York,"Emery, Humberlea",43.724766,-79.532242,Strathburn Park
0,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,Nara Sushi
0,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,Baskin Robbins
0,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,Pizza Pizza
0,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,McDonald's
0,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,Safari Bar and Grill
0,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,Il Fornaro
0,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,Shoppers Drug Mart
0,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,Subway
